In [ ]:
from oneatlas import OneAtlasClient
import json
import geopandas as gpd

# Process overview
- Convert input aoi layer into geojson polygons
- Search API for available images and convert available images footprints to geodataframe
- spatial join the footprints to the aoi gdf
- use the aoi - images gdf to plot image quicklooks
- also convert them to the image windows used to place an order
- place an order for a window around the AOI - if just a 2 km by 2 km or similar this costs magnitudes less than ordering full image

In [ ]:
# Need to look at jpeg2000 output, options to project to BNG etc before order

In [ ]:
# function to convert polygon gdf to bounding box list
aoi_gdf = gpd.read_file("inputs/test_quarries.shp")

In [ ]:
aoi_gdf["geometry"] = aoi_gdf.buffer(1000)

In [ ]:
def aoi_gdf_to_search_geojson(aoi_gdf, id_column=None):
    if id_column is None:
        id_column = "id"
        aoi_gdf[id_column] = aoi_gdf.index
    json_str = aoi_gdf[[id_column, "geometry"]].to_crs(epsg=4326).to_json()
    return json.loads(json_str)

In [ ]:
search_geojson = aoi_gdf_to_search_geojson(aoi_gdf)

In [ ]:
search_geojson

In [ ]:
search_test = search_geojson["features"][0]["geometry"]

In [ ]:
# Read local config.json to get api key
with open("config.json", "r") as file:
    config = json.load(file)


api_key = config["api_key"]

In [ ]:
client = OneAtlasClient(api_key=api_key)

In [ ]:
results = client.search(
    {
        "cloudCover": "[0,30]",
        "incidenceAngle": "[0,40]",
        "processingLevel": "SENSOR",
        "relation": "intersects",
        "geometry": search_test,
    }
)
result = results["features"][1]
# client.download_quicklook_to_file(result, "quicklook.jpg")
client.plot_quicklook(result)

In [ ]:
result["properties"]["platform"]

In [ ]:
results["features"][0]["geometry"]["coordinates"]

In [ ]:
results["features"][0]["properties"]["id"]

In [ ]:
order_body = {
    "kind": "order.product",
    "products": [
        {
            "productType": "bundle",
            "radiometricProcessing": "REFLECTANCE",
            "imageFormat": "image/jp2",
            "crsCode": "urn:ogc:def:crs:EPSG::4326",
            "id": image_id,
            "aoi": bbox_order,
        }
    ],
}
price = client.get_price(order_body)

In [ ]:
bbox_order

In [ ]:
order_body = {
    "kind": "order.product",
    "products": [
        {
            "productType": "bundle",
            "radiometricProcessing": "REFLECTANCE",
            "imageFormat": "image/jp2",
            "crsCode": "urn:ogc:def:crs:EPSG::4326",
            "id": "a62ca420-399f-4a7f-8623-eca3bf0d397b",
            "aoi": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [-3.3416796390244294, 58.2848266233006],
                        [-3.3416796390244294, 58.32134905375606],
                        [-3.272209519926957, 58.32134905375606],
                        [-3.272209519926957, 58.2848266233006],
                        [-3.3416796390244294, 58.2848266233006],
                    ]
                ],
            },
        }
    ],
}
client.get_price(order_body)["price"]

In [ ]:
client.create_order(order_body)

In [ ]:
orders = client.list_orders(kind="order.data.product", status="delivered")

In [ ]:
order = orders["items"][0]
client.download_order_to_file(order, "outputs/order.zip")